In [ ]:
import sys
sys.path
sys.path.append("./keras/lib/python3.4")

# Import libraries and modules
import time
from tqdm import tqdm

import copy

import math

import seaborn as sns

import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
import matplotlib.gridspec as gridspec
import matplotlib.colors as col

from sklearn.metrics import roc_curve, auc
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

from sklearn.preprocessing import MinMaxScaler

import numpy as np
np.random.seed(123)
import numpy.ma as ma

import scipy as sp

import pandas as pd

import theano

from keras import backend as K
K.set_image_dim_ordering('tf')

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Reshape, Input
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import LSTM
from keras.layers.wrappers import TimeDistributed
from keras.models import Model

import os
import gc

raster = 200
crime_type = "E05"
date_from = "2013-06-16"
date_to = "2017-03-08"
dist = 5

# insert main dir
main_dir = ""

In [ ]:
X_train = np.load("X_train_feat.npy")
X_validation = np.load("X_validation_feat.npy")

y_train = np.load("y_train_feat.npy")
y_validation = np.load("y_validation_feat.npy")


In [ ]:
epochs = 1
timesteps = 21
batch_size = timesteps

result_file_dir = "%s/output/python/model_results.csv" % main_dir
data_id = "tm_neigh_data"
        
def train_test_model_save_results_crnn(model, model_id):
    model.fit(X_train,
          y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(X_validation, y_validation))
    
    save_dir_model = "%s/output/python/keras/models/%s_%s.h5" % (main_dir, model_id, data_id)
    model.save(save_dir_model)

    return model

In [ ]:
# Model 15 feat
# number of convolutional filters
n_filters = 8

# convolution filter size
n_conv = 2

# pooling window size
n_pool = 2

model=Sequential();      

model.add(TimeDistributed(Conv2D(n_filters, (n_conv, n_conv), padding='same'), input_shape=X_train.shape[1:]))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(Conv2D(n_filters, (n_conv, n_conv))))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(n_pool, n_pool))))
model.add(TimeDistributed(Dropout(0.15)))

model.add(TimeDistributed(Flatten()))
model.add(TimeDistributed(Dense(512)))                
        
model.add(LSTM(100, return_sequences=True, name="lstm_layer1"));

model.add(TimeDistributed(Dense(100)))
model.add(TimeDistributed(Dense(100)))
model.add(TimeDistributed(Dense(100)))

model.add(TimeDistributed(Dropout(0.15)))
         
model.add(TimeDistributed(Dense(1), name="time_distr_dense_one"))
model.add(TimeDistributed(Activation('sigmoid')))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['binary_accuracy'])

print(model.summary())

trained_model = train_test_model_save_results_crnn(model, 'crnn_15_feat')
print("Done.")

In [ ]:
from keras.models import load_model
load_dir_model = "%s/output/python/keras/models/crnn_15_feat_tm_neigh_data.h5" % (main_dir)
model = load_model(load_dir_model)

X_train = np.load("X_train_feat.npy")

timesteps = 21
image_size = 2*dist +1

In [ ]:
predictions = model.predict(X_train)
predictions = predictions.reshape(predictions.shape[0] * predictions.shape[1])
predictions.shape

In [ ]:
image_size = 2*dist +1

# Show filters
instance = X_train[46000:]
instance = instance.astype('float32').reshape(1, instance.shape[0], timesteps, image_size, image_size)

layers = [12]
get_features = K.function([model.layers[0].input, K.learning_phase()], [model.layers[l].output for l in layers])
feature_maps = get_features([instance,1])

In [ ]:
np.save("crnn_feat_data_3.npy", instance)

In [ ]:
instance = X_train[6000]
instance = instance.astype('float32').reshape(1, 1, timesteps, image_size, image_size)
plt.imshow(instance[0][0][1])
plt.show()